### Imports

In [1]:
import os
import numpy as np
import pandas as pd

### Inputs

In [2]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].iloc[0]

- apmr_edge (Active Prefix Message Ratio): Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das arestas, 
medindo quão bem os campos de prefixo conseguem realizar matching probabilístico para encaminhar mensagens entre nós vizinhos.

- mfsr_edge (Message Forwarding Success Ratio): Proporção de mensagens transmitidas com sucesso através de uma aresta específica, 
considerando interferência, congestionamento e qualidade da conexão sem fio entre dois nós conectados.

- idg_global (Interest Group Delivery): Eficácia global da entrega de mensagens no modo Interest-Group addressing, m
edindo quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham o mesmo interesse da aplicação.

In [3]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

In [4]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

### Fusion

In [5]:
weight_inference_model = 0.50
weight_apmr_edge = 0.20
weight_mfsr_edge = 0.20
weight_idg_global = 0.10

conf = (
    weight_inference_model*inference_output_confidence + 
    weight_apmr_edge*radnet_input_apmr_edge + 
    weight_mfsr_edge*radnet_input_mfsr_edge + 
    weight_idg_global*radnet_input_idg_global
)

In [6]:
tau = 0.5
cls = (conf >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [7]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-55    2.0   17     0.30   1        Bom      0.86      
-85    3.8   30     0.10   1        Bom      0.72      
-35    1.2   5      0.05   1        Bom      0.87      
-88    9.5   95     0.90   1        Bom      0.88      
-40    8.2   85     0.80   1        Bom      0.88      
-87    1.5   8      0.15   1        Bom      0.87      
-62    2.8   32     0.28   1        Bom      0.86      
-58    3.2   28     0.32   1        Bom      0.86      
-52    2.1   20     0.25   1        Bom      0.89      
-48    2.5   25     0.22   1        Bom      0.89      
-50    1.8   22     0.28   1        Bom      0.89      
-75    5.2   65     0.60   1        Bom      0.89      
-78    4.8   70     0.65   1        Bom      0.88      
-72    5.5   60     0.58   1        Bom      0.88      
-45    4.5   15     0.70   1        Bom      0.64      
-70    1.9   45     0.20   1        Bom      0.8

### Save

In [8]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
